# WebScraping - Sofascore Game Stats

In [1]:
# Importando bibliotecas
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re

In [2]:
# link = input('Insira o link da partida no Sofascore \n')
link = 'https://www.sofascore.com/sao-jose-ec-ec-noroeste/mOcsUzK#id:11888598'
# Expressão regular para capturar o ID
pattern = r'id:(\d+)(?!\d)'

# Encontrando o ID
match = re.search(pattern, link)
if match:
    id_value = match.group(1)
else:
    print("ID não encontrado.")

In [3]:
# Criando navegador e entrando no site https://www.sofascore.com/sao-jose-ec-ec-noroeste/mOcsUzK#id:11888598,tab:standings

# driver = webdriver.Chrome(service = Service(ChromeDriverManager()))
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install())) #utilize esta linha caso esteja utilizando o cóodigo pela primeira vez

url = "https://www.sofascore.com/sao-jose-ec-ec-noroeste/mOcsUzK#id:11888598"

driver.get(link)

In [4]:
# Pegando informações da partida
teams = driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[1]/div/div/div[1]/div[1]/bdi''').text.split(' - ')
goals = [driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div/div[1]/span/span[1]''').text,driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div/div[1]/span/span[3]''').text]
formations = [driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div/div[1]/div/div/div[1]/div[1]/div[2]/span''').text,driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div/div[1]/div/div/div[4]/div[1]/div[2]/span''').text]
managers = [driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/div[1]/a/div/div[2]/div/div/span/span''').text,driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/div[2]/div[2]/a/div/div[2]/div/div/span/span''').text]
date_and_time = driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[1]/div[9]/div[1]/bdi''').text.split(', ')
attendance = driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[1]/div[9]/div[2]/bdi''').text
tournament = driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[1]/div/div/div[1]/ul/li[3]/a''').text
# df_starting11 = driver.find_elements(By.CLASS_NAME,"gvZNHo")
# for e in df_starting11:
#     print(e.text)
dict_info_match = {'Teams':teams, 'Goals':goals, 'Formations': formations, 'Date':date_and_time[0], 'Time':date_and_time[1], 'Attendance':attendance,'Tournament':tournament}
dict_info_match

{'Teams': ['São José EC', 'EC Noroeste'],
 'Goals': ['2', '1'],
 'Formations': ['4-2-3-1', '3-5-2'],
 'Date': '17/01/2024',
 'Time': '20:00',
 'Attendance': '4526',
 'Tournament': 'Paulista, Série A2, Round 1'}

In [5]:
estatisticas = driver.find_elements(By.CLASS_NAME, "gMqrdr")
dados = driver.find_elements(By.CLASS_NAME, "hhKEsZ")
if len(estatisticas) * 2 != len(dados):
    raise ValueError("O número de estatísticas não corresponde ao número de dados.")

# Criar um dicionário para armazenar os dados agrupados
data_dict = {estatisticas[i].text: [dados[j].text, dados[j+1].text] for i, j in zip(range(len(estatisticas)), range(0, len(dados), 2))}
data_dict['Team'] = teams
data_dict['Goals'] = goals
data_dict['Formation'] = formations
data_dict['Manager'] = managers
# Criar DataFrame a partir do dicionário
df_estatisticas = pd.DataFrame(data_dict)

# Exibir o DataFrame
df_estatisticas

,Ball possession,Expected Goals (xG),Total shots,Goalkeeper saves,Corner kicks,Fouls,Passes,Tackles,Yellow cards,Red cards,...,Dribbles,Total tackles,Interceptions,Clearances,Total saves,Goal kicks,Team,Goals,Formation,Manager
0,41%,0.46,8,2,1,12,337,7,0,0,...,15/20 (75%),7,44,21,2,9,São José EC,2,4-2-3-1,Ricardo Costa
1,59%,1.42,11,1,7,10,538,1,0,0,...,15/22 (68%),1,31,11,1,5,EC Noroeste,1,3-5-2,Moisés Egert


In [6]:
#Pegando as informações dos jogadores
botao_player_statistics = driver.find_element('xpath','''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[1]/div[2]''')

driver.execute_script('arguments[0].click();',botao_player_statistics)

# Selecionando os dados do time da casa

botao_home_team = driver.find_element('xpath','''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div[2]/div/table/thead/tr/th[2]/div/div[2]/img''')

driver.execute_script('arguments[0].click();',botao_home_team)

lista_de_tabelas = []

i=1

# Clicando no botão, pegando a tabela, e passando para a próxima tabela

for botao in range(0,6):

    botao_tabela = driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div[1]/button[{i}]''')

    driver.execute_script('arguments[0].click();',botao_tabela)

    tabela = driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute('outerHTML')

    tabela_final = pd.read_html(html_tabela)[0]

    lista_de_tabelas.append(tabela_final)
    
    i += 1
    
tabela_gk = lista_de_tabelas[5]

del lista_de_tabelas[5]
del i

prefixos = ['Summary ', 'Attack ', 'Defence ', 'Passing ', 'Duels ', 'Goalkeeper ']
for prefixo, tabela in zip(prefixos, lista_de_tabelas):
    tabela.columns = [prefixo + coluna for coluna in tabela.columns]

df_home = pd.concat(lista_de_tabelas, axis=1, join='inner')

df_home = df_home.drop(columns=['Summary Unnamed: 0', 'Attack Unnamed: 0', 'Attack +','Attack Sofascore rating','Attack Position','Defence Unnamed: 0', 'Defence +','Defence Tackles','Defence Position','Defence Sofascore rating', 'Passing Unnamed: 0', 'Passing +','Passing Position','Passing Accurate passes', 'Passing Sofascore rating','Duels Unnamed: 0','Duels +','Duels Position','Duels Sofascore rating','Duels Duels (won)','Duels Ground duels (won)','Duels Aerial duels (won)'])

df_home = df_home.rename(columns={'Summary +' : 'Name', 
                        'Summary Goals' : 'Goals', 
                        'Summary Assists' : 'Assists',
                        'Summary Tackles' : 'Tackles', 
                        'Summary Accurate passes' : 'Accurate passes', 
                        'Summary Duels (won)' : 'Duels (won)',
                        'Summary Ground duels (won)' : 'Ground duels (won)', 
                        'Summary Aerial duels (won)' : 'Aerial duels (won)',
                        'Summary Minutes played' : 'Minutes played', 
                        'Summary Position' : 'Position',
                        'Summary Sofascore rating' : 'Rating',
                        'Attack Shots on target' : 'Shots on target', 
                        'Attack Expected Goals (xG)' : 'Expected Goals (xG)',
                        'Attack Shots off target' : 'Shots off target', 
                        'Attack Shots blocked' : 'Shots blocked',
                        'Attack Dribble attempts (succ.)' : 'Dribble attempts (succ.)', 
                        'Defence Defensive actions' : 'Defensive actions', 
                        'Defence Clearances' : 'Clearances',
                        'Defence Blocked shots' : 'Blocked shots', 
                        'Defence Interceptions' : 'Interceptions', 
                        'Defence Dribbled past' : 'Dribbled past', 
                        'Passing Touches' : 'Touches',  
                        'Passing Key passes' : 'Key passes',
                        'Passing Crosses (acc.)' : 'Crosses (acc.)', 
                        'Passing Long balls (acc.)' : 'Long balls (acc.)', 
                        'Duels Possession lost' : 'Possession lost', 
                        'Duels Fouls' : 'Fouls committed',
                        'Duels Was fouled' : 'Fouls suffered', 
                        'Duels Offsides' : 'Offsides'
                        })
tabela_gk = tabela_gk.rename(columns={'+':'Name', 'Notes':'GK Notes'})
df_home = pd.merge(df_home,tabela_gk, how='left', on=['Name'])
df_home = df_home.drop(columns=['Unnamed: 0','Sofascore rating'])
df_home['Team'] = teams[0]

In [7]:
# Selecionando os dados do time visitante

botao_away_team = driver.find_element('xpath','''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div[2]/div/table/thead/tr/th[2]/div/div[3]/img''')

driver.execute_script('arguments[0].click();',botao_away_team)

lista_de_tabelas = []

i=1

# Clicando no botão, pegando a tabela, e passando para a próxima tabela

for botao in range(0,6):

    botao_tabela = driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div[1]/button[{i}]''')

    driver.execute_script('arguments[0].click();',botao_tabela)

    tabela = driver.find_element('xpath',f'''/html/body/div[1]/main/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute('outerHTML')

    tabela_final = pd.read_html(html_tabela)[0]

    lista_de_tabelas.append(tabela_final)
    
    i += 1
    
tabela_gk = lista_de_tabelas[5]

del lista_de_tabelas[5]
del i

prefixos = ['Summary ', 'Attack ', 'Defence ', 'Passing ', 'Duels ', 'Goalkeeper ']
for prefixo, tabela in zip(prefixos, lista_de_tabelas):
    tabela.columns = [prefixo + coluna for coluna in tabela.columns]

df_away = pd.concat(lista_de_tabelas, axis=1, join='inner')

df_away = df_away.drop(columns=['Summary Unnamed: 0', 'Attack Unnamed: 0', 'Attack +','Attack Sofascore rating','Attack Position','Defence Unnamed: 0', 'Defence +','Defence Tackles','Defence Position','Defence Sofascore rating', 'Passing Unnamed: 0', 'Passing +','Passing Position','Passing Accurate passes', 'Passing Sofascore rating','Duels Unnamed: 0','Duels +','Duels Position','Duels Sofascore rating','Duels Duels (won)','Duels Ground duels (won)','Duels Aerial duels (won)'])

df_away = df_away.rename(columns={'Summary +' : 'Name', 
                        'Summary Goals' : 'Goals', 
                        'Summary Assists' : 'Assists',
                        'Summary Tackles' : 'Tackles', 
                        'Summary Accurate passes' : 'Accurate passes', 
                        'Summary Duels (won)' : 'Duels (won)',
                        'Summary Ground duels (won)' : 'Ground duels (won)', 
                        'Summary Aerial duels (won)' : 'Aerial duels (won)',
                        'Summary Minutes played' : 'Minutes played', 
                        'Summary Position' : 'Position',
                        'Summary Sofascore rating' : 'Rating',
                        'Attack Shots on target' : 'Shots on target', 
                        'Attack Expected Goals (xG)' : 'Expected Goals (xG)',
                        'Attack Shots off target' : 'Shots off target', 
                        'Attack Shots blocked' : 'Shots blocked',
                        'Attack Dribble attempts (succ.)' : 'Dribble attempts (succ.)', 
                        'Defence Defensive actions' : 'Defensive actions', 
                        'Defence Clearances' : 'Clearances',
                        'Defence Blocked shots' : 'Blocked shots', 
                        'Defence Interceptions' : 'Interceptions', 
                        'Defence Dribbled past' : 'Dribbled past', 
                        'Passing Touches' : 'Touches',  
                        'Passing Key passes' : 'Key passes',
                        'Passing Crosses (acc.)' : 'Crosses (acc.)', 
                        'Passing Long balls (acc.)' : 'Long balls (acc.)', 
                        'Duels Possession lost' : 'Possession lost', 
                        'Duels Fouls' : 'Fouls committed',
                        'Duels Was fouled' : 'Fouls suffered', 
                        'Duels Offsides' : 'Offsides'
                        })
tabela_gk = tabela_gk.rename(columns={'+':'Name', 'Notes':'GK Notes'})
df_away = pd.merge(df_away,tabela_gk, how='left', on=['Name'])
df_away = df_away.drop(columns=['Unnamed: 0','Sofascore rating'])
df_away['Team'] = teams[1]

In [8]:
dados_jogadores = base_final = pd.concat([df_home, df_away])
base_final['Match ID'] = id_value

In [9]:
base_final

,Name,Goals,Assists,Tackles,Accurate passes,Duels (won),Ground duels (won),Aerial duels (won),Minutes played,Position,...,Fouls suffered,Offsides,Saves,Goals prevented,Punches,Runs out (succ.),High claims,GK Notes,Team,Match ID
0,Leonardo Pereira,0,1,1,40/49 (82%),10 (7),7 (5),3 (2),90',D,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,São José EC,11888598
1,Nicholas,1,0,0,19/23 (83%),10 (3),10 (3),0 (0),68',M,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,São José EC,11888598
2,João Gabriel,1,0,0,17/19 (89%),12 (3),9 (2),3 (1),79',M,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,São José EC,11888598
3,Matheus Serafim,0,0,0,13/23 (57%),31 (15),31 (15),0 (0),90',M,...,2,0,NaN,NaN,NaN,NaN,NaN,NaN,São José EC,11888598
4,Caio Cesár,0,0,0,36/40 (90%),16 (9),7 (1),9 (8),90',D,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,São José EC,11888598
5,Diogo Marzagão,0,0,2,29/40 (73%),14 (5),10 (2),4 (3),90',M,...,2,0,NaN,NaN,NaN,NaN,NaN,NaN,São José EC,11888598
6,Wagner Balotelli,0,0,1,25/33 (76%),17 (9),13 (7),4 (2),90',M,...,2,0,NaN,NaN,NaN,NaN,NaN,NaN,São José EC,11888598
7,Ariel Luciano do Nascimento,0,0,0,8/10 (80%),0 (0),0 (0),0 (0),90',G,...,0,0,2.0,NaN,0.0,0 (0),0.0,Saves from inside box: 1,São José EC,11888598
8,Rafael Verrone,0,0,0,33/39 (85%),12 (3),9 (1),3 (2),90',D,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,São José EC,11888598
9,Léo Gobo,0,0,0,35/41 (85%),14 (7),10 (4),4 (3),90',D,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,São José EC,11888598


In [11]:
driver.quit()